In [1]:
import random
import pandas as pd
import datetime

# Step 1: Generate Users
def generate_users(num_users):
    users = []
    for user_id in range(1, num_users + 1):
        username = f"user_{user_id}"
        password = f"pass_{user_id}"
        users.append([user_id, username, password])
    return pd.DataFrame(users, columns=['user_id', 'username', 'password'])

# Function to classify users into active and less active
def classify_users(df_users):
    active_users = df_users.sample(frac=0.2).user_id.tolist()  # 20% of active users
    less_active_users = df_users[~df_users['user_id'].isin(active_users)].user_id.tolist()  # 80% less active
    return active_users, less_active_users

# Function to generate interactions based on active and less active users
def generate_interactions(num_interactions, df_movies, df_users, active_users, less_active_users, emotions):
    interactions = []
    interaction_history = {}

    # Ratio: for every 6 interactions, 5 will be 'shown' and 1 will be 'view'
    view_ratio = 1 / 6
    shown_ratio = 5 / 6

    for _ in range(num_interactions):
        # Select an active or less active user
        if random.random() > 0.3:
            user_id = random.choice(active_users)
        else:
            user_id = random.choice(less_active_users)

        # Randomly select an emotion
        emotion = random.choice(emotions)

        # Filter movies that match the selected emotion
        filtered_movies = df_movies[df_movies['emotions'].apply(lambda x: emotion in x)]

        # Proceed only if there are movies that match the selected emotion
        if not filtered_movies.empty:
            # Randomly select a movie from the filtered list
            movie_id = filtered_movies.sample(1).index[0]

            # Initialize interaction history for the user if it doesn't exist
            if user_id not in interaction_history:
                interaction_history[user_id] = {}

            # Initialize interaction history for the movie if it doesn't exist
            if movie_id not in interaction_history[user_id]:
                interaction_history[user_id][movie_id] = []

            # Determine if this interaction will be "shown" or "view" based on the ratio
            if len(interaction_history[user_id][movie_id]) == 0:
                if random.random() < shown_ratio:
                    interaction_type = 'shown'
                else:
                    interaction_type = 'view'
            else:
                continue  # If there's already an interaction for that movie, skip

            # Generate a random date within the last year
            date = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365))

            # Add interaction to the list
            interactions.append([user_id, movie_id, emotion, interaction_type, date])

            # Register the interaction to avoid duplicates
            interaction_history[user_id][movie_id].append(interaction_type)

    return pd.DataFrame(interactions, columns=['user_id', 'movie_id', 'emotion', 'interaction_type', 'date'])

# Function to generate favorites based on 'view' interactions
def generate_favorites(df_interactions):
    favorites = []

    # Only select 'view' interactions to be added as favorites
    viewed_interactions = df_interactions[df_interactions['interaction_type'] == 'view']

    # Select 30% of 'view' interactions
    num_favorites = int(len(viewed_interactions) * 0.5)

    for _, selected_interaction in viewed_interactions.sample(num_favorites).iterrows():
        user_id = selected_interaction['user_id']
        movie_id = selected_interaction['movie_id']
        view_date = selected_interaction['date']
        
        # The date the movie is marked as favorite must be on or after the view date
        days_later = random.randint(0, (datetime.datetime.now() - view_date).days)
        date_added = view_date + datetime.timedelta(days=days_later)

        favorites.append([user_id, movie_id, date_added])

    return pd.DataFrame(favorites, columns=['user_id', 'movie_id', 'date_added'])

# Function to generate ratings based on 50% of the favorites
def generate_ratings(df_favorites):
    ratings = []

    # Select only 50% of the movies marked as favorites
    num_ratings = int(len(df_favorites) * 0.7)

    for _, selected_favorite in df_favorites.sample(num_ratings).iterrows():
        user_id = selected_favorite['user_id']
        movie_id = selected_favorite['movie_id']
        favorite_date = selected_favorite['date_added']
        
        # The rating date must be on or after the date the movie was added to favorites
        days_later = random.randint(0, (datetime.datetime.now() - favorite_date).days)
        date_rated = favorite_date + datetime.timedelta(days=days_later)
        
        # Assign a rating between 1 and 10, with a bias towards higher ratings
        rating = random.choices(range(1, 11), weights=[1, 1, 2, 2, 3, 3, 4, 4, 5, 5], k=1)[0]
        
        # Add the rating
        ratings.append([user_id, movie_id, rating, date_rated])

    return pd.DataFrame(ratings, columns=['user_id', 'movie_id', 'rating', 'date'])

In [2]:
# Load the movie dataset
df_movies = pd.read_csv('../data/imdb_clean.csv')  # Ensure the CSV file is available
df_movies.index = df_movies.index + 1  # Ensure the index starts at 1 to simulate movie_id

# Create a DataFrame of users with usernames and passwords
num_users = 1000
df_users = generate_users(num_users)

# Define available emotions
emotions = ['Happy', 'Sad', 'Excited', 'Relaxed', 'Scared', 'Inspired', 'Sweet']

# Classify users into active and less active
active_users, less_active_users = classify_users(df_users)

# Generate interactions
df_interactions = generate_interactions(50000, df_movies, df_users, active_users, less_active_users, emotions)

# Generate favorites (30% of the viewed movies)
df_favorites = generate_favorites(df_interactions)

# Generate ratings (50% of the favorite movies)
df_ratings = generate_ratings(df_favorites)

# Display generated data
print(df_users.head())
print(df_interactions.head())
print(df_favorites.head())
print(df_ratings.head())

   user_id username password
0        1   user_1   pass_1
1        2   user_2   pass_2
2        3   user_3   pass_3
3        4   user_4   pass_4
4        5   user_5   pass_5
   user_id  movie_id   emotion interaction_type                       date
0      173      7386   Excited            shown 2024-07-23 20:06:29.668869
1      488      4990   Excited            shown 2024-08-20 20:06:29.682714
2      116      8957     Sweet            shown 2024-06-13 20:06:29.683321
3      246      4395    Scared            shown 2024-05-12 20:06:29.683321
4      497      7210  Inspired            shown 2024-07-20 20:06:29.683321
   user_id  movie_id                 date_added
0      897      2481 2024-08-16 20:10:25.783521
1      286      3291 2024-01-24 20:06:50.667913
2      714      2731 2024-10-09 20:09:20.134275
3      183       206 2024-10-16 20:06:38.566719
4      231      9072 2024-08-10 20:08:16.620272
   user_id  movie_id  rating                       date
0      953      5284       9 202

In [3]:
# Guardar los datos generados en CSV
df_users.to_csv('../data/users.csv', index=False)
df_interactions.to_csv('../data/interactions.csv', index=False)
df_favorites.to_csv('../data/favorites.csv', index=False)
df_ratings.to_csv('../data/ratings.csv', index=False)

In [4]:
import mysql.connector

# Conectar a la base de datos MySQL
conn = mysql.connector.connect(
    host="localhost",  
    user="root",  
    password="123456",  
    database="movie_recommendations"
)
c = conn.cursor()

In [5]:
# Insertion into the database (if you want to work directly in MySQL)
def insert_users(df_users):
    for _, row in df_users.iterrows():
        query = """
        INSERT INTO users (user_id, username, password) 
        VALUES (%s, %s, %s)
        """
        values = (row['user_id'], row['username'], row['password'])
        c.execute(query, values)
    conn.commit()

def insert_interactions(df_interactions):
    for _, row in df_interactions.iterrows():
        query = """
        INSERT INTO interactions (user_id, movie_id, emotion, interaction_type, date) 
        VALUES (%s, %s, %s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['emotion'], row['interaction_type'], row['date'])
        c.execute(query, values)
    conn.commit()

def insert_favorites(df_favorites):
    for _, row in df_favorites.iterrows():
        query = """
        INSERT INTO favorites (user_id, movie_id, date_added) 
        VALUES (%s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['date_added'])
        c.execute(query, values)
    conn.commit()

def insert_ratings(df_ratings):
    for _, row in df_ratings.iterrows():
        query = """
        INSERT INTO ratings (user_id, movie_id, rating, date) 
        VALUES (%s, %s, %s, %s)
        """
        values = (row['user_id'], row['movie_id'], row['rating'], row['date'])
        c.execute(query, values)
    conn.commit()

# Call the insertion functions
insert_users(df_users)
insert_interactions(df_interactions)
insert_favorites(df_favorites)
insert_ratings(df_ratings)

# Close the database connection
conn.close()

In [6]:
user_id = 1

# Filter interactions for the user
user_interactions = df_interactions[df_interactions['user_id'] == user_id]
print("User interactions:")
print(user_interactions)

# Filter favorites for the user
user_favorites = df_favorites[df_favorites['user_id'] == user_id]
print("\nUser favorites:")
print(user_favorites)

# Filter ratings for the user
user_ratings = df_ratings[df_ratings['user_id'] == user_id]
print("\nUser ratings:")
print(user_ratings)

User interactions:
       user_id  movie_id   emotion interaction_type                       date
2461         1      6342     Happy            shown 2024-05-05 20:06:43.450184
6191         1      5566  Inspired            shown 2024-05-18 20:07:04.050256
6585         1      6029     Happy            shown 2023-12-18 20:07:06.066529
8143         1      2708     Sweet            shown 2024-04-27 20:07:16.435538
10150        1      6842     Happy            shown 2024-03-04 20:07:28.649302
10822        1      7958   Relaxed            shown 2024-09-18 20:07:33.253960
12394        1      3489     Sweet            shown 2024-10-19 20:07:43.334916
15812        1      2735     Happy            shown 2024-06-30 20:08:03.167846
17837        1      1609  Inspired            shown 2024-09-17 20:08:15.284659
18809        1      4046     Sweet            shown 2024-07-05 20:08:20.472608
21868        1      6436   Relaxed            shown 2024-06-26 20:08:38.267351
23291        1       413     Happ

In [7]:
df_users

,user_id,username,password
0,1,user_1,pass_1
1,2,user_2,pass_2
2,3,user_3,pass_3
3,4,user_4,pass_4
4,5,user_5,pass_5
...,...,...,...
995,996,user_996,pass_996
996,997,user_997,pass_997
997,998,user_998,pass_998
998,999,user_999,pass_999


In [8]:
df_interactions

,user_id,movie_id,emotion,interaction_type,date
0,173,7386,Excited,shown,2024-07-23 20:06:29.668869
1,488,4990,Excited,shown,2024-08-20 20:06:29.682714
2,116,8957,Sweet,shown,2024-06-13 20:06:29.683321
3,246,4395,Scared,shown,2024-05-12 20:06:29.683321
4,497,7210,Inspired,shown,2024-07-20 20:06:29.683321
...,...,...,...,...,...
42670,188,8073,Scared,shown,2024-09-02 20:10:42.449590
42671,725,3455,Scared,shown,2024-04-07 20:10:42.467330
42672,378,1898,Excited,shown,2024-07-10 20:10:42.467330
42673,608,3503,Excited,shown,2024-01-16 20:10:42.467330


In [9]:
df_favorites

,user_id,movie_id,date_added
0,897,2481,2024-08-16 20:10:25.783521
1,286,3291,2024-01-24 20:06:50.667913
2,714,2731,2024-10-09 20:09:20.134275
3,183,206,2024-10-16 20:06:38.566719
4,231,9072,2024-08-10 20:08:16.620272
...,...,...,...
3513,853,9307,2024-05-14 20:09:27.467404
3514,714,7097,2024-09-16 20:09:36.250744
3515,608,54,2024-10-18 20:07:56.920784
3516,78,126,2024-02-10 20:09:20.536064


In [10]:
df_ratings

,user_id,movie_id,rating,date
0,953,5284,9,2024-09-26 20:06:30.200527
1,259,8292,1,2024-09-25 20:08:48.784320
2,642,1169,3,2024-10-15 20:06:32.735592
3,292,6859,10,2024-09-16 20:09:11.250483
4,107,4571,3,2024-07-14 20:07:15.739259
...,...,...,...,...
2457,330,8820,9,2024-10-18 20:07:04.035248
2458,340,8763,7,2024-10-07 20:09:20.451634
2459,150,5523,10,2024-08-02 20:08:36.853663
2460,921,5035,5,2024-07-04 20:06:45.019658
